# DRILL: Prepare the Data

[Download the Excel file here](https://ucr.fbi.gov/crime-in-the-u.s/2013/crime-in-the-u.s.-2013/tables/table-8/table-8-state-cuts/table_8_offenses_known_to_law_enforcement_new_york_by_city_2013.xls) on crime data in New York State in 2013, provided by the FBI: UCR ([Thinkful mirror](https://raw.githubusercontent.com/Thinkful-Ed/data-201-resources/master/New_York_offenses/NEW_YORK-Offenses_Known_to_Law_Enforcement_by_City_2013%20-%2013tbl8ny.csv)).

Prepare this data to model with multivariable regression (including data cleaning if necessary) according to this specification:

$$ Property crime = \alpha + Population + Population^2 + Murder + Robbery$$

The 'population' variable is already set for you, but you will need to create the last three features.  Robbery and Murder are currently continuous variables.  For this model, please use these variables to create  categorical features where values greater than 0 are coded 1, and values equal to 0 are coded 0.  You'll use this data and model in a later assignment- for now, just write the code you need to get the data ready.  Don't forget basic data cleaning procedures, either!  Do some graphing to see if there are any anomalous cases, and decide how you want to deal with them.


In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from sklearn import linear_model
%matplotlib inline
import seaborn as sns

pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

In [2]:
df = pd.read_excel("table_8_offenses_known_to_law_enforcement_new_york_by_city_2013.xls", skiprows=4,skip_footer=3)
df.head()

C:\Users\Ruecker-PC\Anaconda3\lib\site-packages\pandas\util\_decorators.py:177: FutureWarning: the 'skip_footer' keyword is deprecated, use 'skipfooter' instead
  return func(*args, **kwargs)


,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3
0,Adams Village,1861,0,0,nan,0,0,0,12,2,10,0,0.000
1,Addison Town and Village,2577,3,0,nan,0,0,3,24,3,20,1,0.000
2,Akron Village,2846,3,0,nan,0,0,3,16,1,15,0,0.000
3,Albany,97956,791,8,nan,30,227,526,4090,705,3243,142,nan
4,Albion Village,6388,23,0,nan,3,4,16,223,53,165,5,nan


## Data exploration

In [3]:
df.describe()

,Population,Violent crime,Murder and nonnegligent manslaughter,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3
count,348.000,348.000,348.000,0.000,348.000,348.000,348.000,348.000,348.000,348.000,348.000,187.000
mean,40037.632,201.595,1.566,nan,5.865,72.902,121.261,792.606,119.684,637.017,35.905,1.872
std,450037.368,2815.269,18.304,nan,60.425,1031.033,1706.132,7659.725,924.949,6346.054,403.424,10.693
min,526.000,0.000,0.000,nan,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,3003.000,2.000,0.000,nan,0.000,0.000,1.000,40.500,6.000,31.000,0.000,0.000
50%,7233.500,6.000,0.000,nan,0.000,1.000,4.000,112.500,17.500,94.000,2.000,0.000
75%,18427.500,22.000,0.000,nan,2.000,5.000,14.000,341.000,51.250,287.250,7.000,1.000
max,8396126.000,52384.000,335.000,nan,1112.000,19170.000,31767.000,141971.000,16606.000,117931.000,7434.000,132.000


In [4]:
#print('Cities:\n', df['City'].unique())
print('Total number of cities:', len(df['City'].unique()))

Total number of cities: 348


In [5]:
# Check how many data is missing
def check_nans(df):
    for column in df.columns.unique():
        print(column,":",df[column].isnull().sum()/df.shape[0]*100 ,"% is Nan")

check_nans(df)

City : 0.0 % is Nan
Population : 0.0 % is Nan
Violent
crime : 0.0 % is Nan
Murder and
nonnegligent
manslaughter : 0.0 % is Nan
Rape
(revised
definition)1 : 100.0 % is Nan
Rape
(legacy
definition)2 : 0.0 % is Nan
Robbery : 0.0 % is Nan
Aggravated
assault : 0.0 % is Nan
Property
crime : 0.0 % is Nan
Burglary : 0.0 % is Nan
Larceny-
theft : 0.0 % is Nan
Motor
vehicle
theft : 0.0 % is Nan
Arson3 : 46.264367816091955 % is Nan


In [6]:
# Clean the data set
# Remove the new lines in the column names
df1 = df
df1.columns= df1.columns.str.replace('\n',' ')
# Remove left and right white space from towns
df1.City = df1.City.str.lstrip()
df1.City = df1.City.str.rstrip()
# Remove Rape (revised definition)1
df1 = df1.drop(['Rape (revised definition)1'],axis=1)
df1.head()

#Remove all nan 
df2 = df1.dropna()
# How much data remains?
print("Complete datasets:",df2.shape[0])

Complete datasets: 187


In [ ]:
# Check for outliers
df2s = pd.melt(df2, id_vars=['City'])
df2s.head()
g = sns.FacetGrid(df2s, col="variable",sharey=False,sharex=False, col_wrap=5, size=5, aspect=.5)
g = (g.map(sns.boxplot, "value"))
g=g.set(xscale="log")

C:\Users\Ruecker-PC\Anaconda3\lib\site-packages\seaborn\axisgrid.py:703: UserWarning: Using the boxplot function without specifying `order` is likely to produce an incorrect plot.
  warnings.warn(warning)


In [ ]:
# Make a scatterplot matrix.
g = sns.PairGrid(df2.dropna(), diag_sharey=False)
# Scatterplot.
g.map_upper(plt.scatter, alpha=.5)
# Fit line summarizing the linear relationship of the two variables.
g.map_lower(sns.regplot, scatter_kws=dict(alpha=0))
# Give information about the univariate distributions of the variables.
g.map_diag(sns.kdeplot, lw=3)
plt.show()

All variables are highly correlated.
But the distribution of most variables are not normal, because of high outliers. But those high outliears are matched by a few cities with higher population. Therefore, I won't remove any outliers.

In [ ]:
# Sort data by population
df2.sort_values(by=['Population'], ascending =False).head()

In [ ]:
# Remove outliers
#Assuming very low crimes are possible.

#def remove_outliers(df):
#    for column in df.columns:
#        df[column] = pd.to_numeric(df[column])
#        P75 = np.percentile(df[column], 75)
        #print(P75)
#        Median = np.median(df[column])
#        print('P75:',column, P75)
#        print('Max before:', df[column].max())
#        df[column]= np.where(df[column]>(P75*1.5), Median , df[column]) # Remove high outliers
#        print('Max after:', df[column].max())

#df3 = remove_outliers(df2.iloc[:,1:] )# Use onlt the numerical data
#df3['City'] = df2['City']

## Generate the new features

In [ ]:
# Generate new features based on the instruction of the assignment
df2['Population^2']=df2['Population']**2
df2['Murder_Bi']=np.where(df2['Murder and nonnegligent manslaughter']>0, 1, 0)
df2['Robbery_Bi']=np.where(df2['Robbery']>0, 1, 0)
df2.head()